In [2]:
import pandas as pd

In [3]:
genre = "horror"
min_year = 2010
max_year = 2020

In [7]:
# drop rows with wrong format for the date null values

missing_values = [""]
df = pd.read_csv("result/" + genre + "/" + genre + "RatingsRaw.csv", na_values = missing_values)
df = df.drop(df[df['date'] == '2'].index)

# sort the date to be in ascending order
df['date'] = df['date'].astype('datetime64[ns]')

# sort the date of each movie to be in ascending order
df.sort_values(by=['film_id','date'], inplace=True)

# make a new dataFrame and convert it to csv for each movie with the data of its ratings mean
df1 = df.drop(columns = ['date'])
df1 = df1.groupby(by=['film_id']).mean()
df1.to_csv("result/" + genre + "/" + genre + "RatingsAverage.csv")
df1 = pd.read_csv("result/" + genre + "/" + genre + "RatingsAverage.csv")
df1.columns = ['film_id', 'average']
df1.to_csv("result/" + genre + "/" + genre + "RatingsAverage.csv", index = False)

# group each movie by the id and date
df = df.groupby(by=["film_id", "date"]).mean()

df.to_csv("result/" + genre + "/" + genre + "RatingsSorted.csv")

# make a new csv files only for those movie with the available review data from 2015 - 2021.
df = pd.read_csv("result/" + genre + "/" + genre + "RatingsSorted.csv")
df['date'] = df['date'].astype('datetime64[ns]')
df['year'] = df['date'].dt.year

applicable_id = []
individual_group = []

grouped_df = df.groupby('film_id')
for key,item in grouped_df:
  individual_group.append(grouped_df.get_group(key))

# iterate for each dataframe in the grouped variable
years = [i for i in range(min_year, max_year + 1)]
index = 0

# create all variable for the years and set it all to False
for i in range(len(individual_group)):
    for year in years:
        exec(f"there_is_{year} = False")

    there_is_missing = False

    # iterate for each row in the dataFrame for the location of each id
    for j in range(len(individual_group[i])):
        test_year = int(individual_group[i].loc[index, "year"])
        try:
            exec(f"there_is_{test_year} = True")
        except:
            pass
        index += 1

    for y in years:
        # if one or more of the year variables are False then change the there_is_missing to True
        exec(f"if (~there_is_{y}): there_is_missing = True")

    # if there are maching data for the analysis append the id to the applicable id list
    if (~there_is_missing):
        applicable_id.append(individual_group[i].loc[index-1, "film_id"])

applicable_id = sorted(applicable_id)

# removing all movie data that doesnt have ratings below min_year
df = df[df['film_id'].isin(applicable_id)]
df = df.drop(df[df['year'] < min_year].index)
df = df.drop(df[df['year'] > max_year].index)
df = df.drop(columns = ["year"])
df.to_csv("result/" + genre + "/" + genre + "RatingsSorted.csv", index = False)

df = pd.read_csv("result/" + genre + "/" + genre + "RatingsSorted.csv")
df['date'] = df['date'].astype('datetime64[ns]')
df['month'] = df['date'].dt.month
df['year'] = df['date'].dt.year
df = df.groupby(by=["film_id", "year", "month"]).mean()

df.to_csv("result/" + genre + "/" + genre + "RatingsSorted.csv")

# merge the average dataFrame with the Sorted dataFrame
df = pd.read_csv("result/" + genre + "/" + genre + "RatingsSorted.csv")
average_df = pd.read_csv("result/" + genre + "/" + genre + "RatingsAverage.csv")
df = pd.merge(df, average_df, on = 'film_id')

# make a final analysis file
df['rating_diff'] = ((df['rating'] - df['average']) / df['average']) * 100
df = df.groupby(by=["month", "year"]).mean()
df = df.drop(columns=['film_id', 'rating', 'average'])
df.sort_values(by=['year','month'], inplace=True)
df.to_csv("result/" + genre + "/" + genre + "AnalysisData.csv")
